# 1st step, calculate all rates

In [1]:
from Er_inf import *
from Er_Energy import *
from Er_EnergyLevelDiagram import *
from Er_RateCalculation import *
from Er_adjustable_parameter import *

Successfully import:

Er_energy
Er_RME
Er_omega

Successfully import: 

sorted_wavelength(energy_dict)
find_ED_transition(energy_dict)
find_MD_transition(energy_dict)

Successfully import:

class Er_EnergyLevelDiagram(self, ion_name, energy_dict)
class Er_EnergyLevelDiagramArrow(self, ion_name, energy_dict, ED_transitions, MD_transitions)

Successfully import:

ED_cal(energy_dict, omega, RME_square, n)
MD_cal(energy_dict, n)
MPR_cal(energy_dict, W0, alpha, phonon)



## 1. Input materials parameters

In [2]:
# Er_energy = {'E0(4I_15_2)':0, 'E1(4I_13_2)': 6632, 'E2(4I_11_2)': 10230, 'E3(4I_9_2)':12553, 'E4(4F_9_2)':15306, 'E5(4S_3_2)':18448, 'E6(2H2_11_2)': 19246, 'E7(4F_7_2)':20497, 'E8(4F_5_2)':22282, 'E9(4F_3_2)':22677, 'E10(2G1_9_2)':24475,
#                 'E11(4G_11_2)':26376, 'E12(4G_9_2)':27319, 'E13(2K_15_2)':27584, 'E14(2G1_7_2)':27825, 'E15(2P_3_2)':31414}

# Er_RME_15={
#        'E1E0':[0.0195, 0.1173, 1.4316],
#        'E2E1':[0.0331, 0.1708, 1.0864],'E2E0':[0.0282, 0.0003, 0.3953],
#        'E3E2':[0.0030, 0.0674, 0.1271],'E3E1':[0.0004, 0.0106, 0.7162],'E3E0':[0, 0.1732, 0.0099],
#        'E4E3':[0.1279, 0.0059, 0.0281],'E4E2':[0.0704, 0.0112, 1.2839],'E4E1':[0.0101, 0.1533, 0.0714],'E4E0':[0, 0.5354, 0.4619],
#        'E5E4':[0, 0.0003, 0.0264],'E5E3':[0, 0.0788, 0.2542],'E5E2':[0, 0.0042, 0.0739],'E5E1':[0, 0, 0.3462],'E5E0':[0, 0, 0.2211],
#        'E6E5':[0, 0.1988, 0.0101],'E6E4':[0.3629, 0.0224, 0.0022],'E6E3':[0.2077, 0.0662, 0.2858],'E6E2':[0.0357, 0.1382, 0.0371],'E6E1':[0.0230, 0.0611, 0.0527],'E6E0':[0.7125, 0.4123, 0.0925],
#        'E7E6':[0.1229, 0.0153, 0.4017],'E7E5':[0.0001, 0.0058, 0],'E7E4':[0.0121, 0.0342, 0.0151],'E7E3':[0.0163, 0.0954, 0.4277],'E7E2':[0.0035, 0.2648, 0.1515],'E7E1':[0, 0.3371, 0.0001],'E7E0':[0, 0.1468, 0.6266],
#        'E8E7':[0.0765, 0.0503, 0.1015],'E8E6':[0, 0.0586, 0.1825],'E8E5':[0.0082, 0.0040, 0],'E8E4':[0.0004, 0.2415, 0.3575],'E8E3':[0.0107, 0.0576, 0.1020],'E8E2':[0, 0.0979, 0.0028],'E8E1':[0, 0.1783, 0.3429],'E8E0':[0, 0, 0.2233],
#        'E9E8':[0.0618, 0.0350, 0],'E9E7':[0.0028, 0.0584, 0],'E9E6':[0, 0.0005, 0.0030],'E9E5':[0.0260, 0, 0],'E9E4':[0, 0.0040, 0.0595],'E9E3':[0, 0.2299, 0.0558],'E9E2':[0, 0.0927, 0.4861],'E9E1':[0, 0, 0.0345],'E9E0':[0, 0, 0.1272],
#        'E10E9':[0, 0.0208, 0.0087],'E10E8':[0.0124,	0.0259,	0.0063],'E10E7':[0.1058, 0.0488, 0.0240],'E10E6':[0.0308, 0.1828, 0.0671],'E10E5':[0, 0.0019, 0.0025],'E10E4':[0.0055, 0.0314, 0.0369],'E10E3':[0.0147, 0.0062, 0.0043],'E10E2':[0.0428, 0.0824, 0.1128],'E10E1':[0.0780, 0.1194, 0.3535],'E10E0':[0, 0.0190, 0.2255],
#        'E11E10':[0.2906, 0.1170, 0.1328],'E11E9':[0, 0.0234, 0.0923],'E11E8':[0, 0.0378, 0.0815],'E11E7':[0.0877, 0.1287, 0.0159],'E11E6':[0.0004, 0.1539, 0.0494],'E11E5':[0, 0.1302, 0.0044],'E11E4':[0.4252, 0.0368, 0.0122],'E11E3':[0.0716, 0.0131, 0.0235],'E11E2':[0.0003, 0.0496, 0.0134],'E11E1':[0.1013, 0.2651, 0.2594],'E11E0':[0.9181, 0.5261, 0.1171],
#        'E12E11':[0.0005, 0.2021, 0.1639],'E12E10':[0.0269, 0, 0.0452],'E12E9':[0, 0.1710, 0.1089],'E12E8':[0.1630, 0.0824, 0.0028],'E12E7':[0.6062, 0.0088, 0.1243],'E12E6':[0.0218, 0.3274, 0.1495],'E12E5':[0, 0.1651, 0.0100],'E12E4':[0.2201, 0.3121, 0.3765],'E12E3':[0.0051, 0.0042, 0.0027],'E12E2':[0.0894, 0.1524, 0.0144],'E12E1':[1.0908, 0.3520, 0.0160],'E12E0':[0, 0.2415, 0.1234],
#        'E13E12':[0, 0.0114, 0.0598],'E13E11':[0.0965, 0.0595, 0.6706],'E13E10':[0, 0.7106, 0.0758],'E13E9':[0, 0, 0.0001],'E13E8':[0, 0, 0.0461],'E13E7':[0, 0.0001, 0.0002],'E13E6':[0.0977, 0.0001, 1.1458],'E13E5':[0, 0, 0.0032],'E13E4':[0, 0.0776, 0.0125],'E13E3':[0, 0.2221, 0.1003],'E13E2':[0.0468, 0.0018, 0.2488],'E13E1':[0.0001, 0.0016, 0.0261],'E13E0':[0.0219, 0.0041, 0.0757],
#        'E14E13':[0, 0.1154, 0.0026],'E14E12':[0.0041, 0.1891, 0.1582],'E14E11':[0.0150, 0.0604, 0.0193],'E14E10':[0.0145, 0.0056, 0.0205],'E14E9':[0.0941, 0.0314, 0],'E14E8':[0.3716, 0.0023, 0.0378],'E14E7':[0.1239, 0.0424, 0.0071],'E14E6':[0.0019, 0.0344, 0.2672],'E14E5':[0.0445, 0.1594, 0],'E14E4':[0.0003, 0.0078, 0.0128],'E14E3':[0.1586, 0.3607, 0.2204],'E14E2':[0.4934, 0.2708, 0.1674],'E14E1':[0, 0.1009, 0.0312],'E14E0':[0, 0.0174, 0.1163],
#        'E15E14':[0.0125, 0.0004, 0], 'E15E13':[0, 0, 0.0268],'E15E12':[0, 0.0125, 0.0053],'E15E11':[0, 0.0266, 0.0107],'E15E10':[0, 0.2083, 0.2591],'E15E9':[0.0123, 0, 0],'E15E8':[0.0173, 0.0433, 0],'E15E7':[0.0211, 0.0076, 0],'E15E6':[0, 0.0168, 0.0263],'E15E5':[0.0813, 0, 0],'E15E4':[0, 0.0464, 0.0060],'E15E3':[0, 0.0461, 0.0041],'E15E2':[0, 0.0995, 0.0400],'E15E1':[0, 0, 0.1478],'E15E0':[0, 0, 0.0172],
#        }



### Have a check

In [2]:
print(len(Er_energy), len(Er_RME))

38 703


## 2. Merge all rates together

In [3]:
# manually input for Yb
tag_Yb = {'c0':7e-39, # Yb-Yb resonant energy transfer
          'Ws': 834,} # Yb ED+MD


tag_Er_ED = ED_cal(Er_energy, Er_omega, Er_RME, n)
tag_Er_MD = MD_cal(Er_energy, n)
tag_Er_mpr = MPR_cal(Er_energy, W0, alpha, E_phonon)
tag_default = tag_Er_ED


for key in tag_Yb:
    tag_default[key] = tag_Yb[key]

# MD+ED
for key in tag_Er_MD:
    tag_default[key] = tag_default[key] + tag_Er_MD[key]

# MD+ED+MPR
for key in tag_Er_mpr:
    tag_default[key] = tag_default[key] + tag_Er_mpr[key]


# tag_default


In [4]:
len(tag_default)

705

# Progress file

In [5]:
import os
import pickle
from datetime import datetime
from time import sleep


# load or initialize progress
def load_or_initialize_progress(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'rb') as f:
            return pickle.load(f)
    else:
        return {}

# save progress
def save_progress(file_path, data):
    with open(file_path, 'wb') as f:
        pickle.dump(data, f)

progress_folder = 'Chuanyu_data_files'
if not os.path.exists(progress_folder):
    os.makedirs(progress_folder)

# unique filename
current_date = datetime.now().strftime('%m_%d_%Y')
base_filename = f'Progress_{current_date}'
index = 1
progress_filepath = os.path.join(progress_folder, f'{base_filename}_{index}.ipynb')
    
while os.path.exists(progress_filepath):
    index += 1
    progress_filepath = os.path.join(progress_folder, f'{base_filename}_{index}.ipynb')


# The first time to run the code, the folder 'Chuanyu_data_files' will have a new file: 'myC_progress.pkl'
# if there is no at least one combination finished, there would be no 'myC_progress.pkl' in the directory
# check the progress data if the program is interruptted


"""
file_path = 'Chuanyu_data_files/myC_progress.pkl'

with open(file_path, 'rb') as file:
    progress_data = pickle.load(file)

for key, value in progress_data.items():
    print(key, value)
"""

# Load or initialize progress data
# if progress_file_path doesn't exist, then return a {}
# if progress_file_path exists, then return a progress_data, which is an incomplete myC
# after interruption, run code and the the progress bar will start from the next combination
# after finish, the progress file is large: ~60M

myC = load_or_initialize_progress(progress_filepath)


# Instruction file

In [6]:
from Er_insturction_file import *

create_instruction_notebook('Chuanyu_data_files', 'SaturationCurves_Er.py', 'OptimalPercentage_Er.py', 'PopulationEvolution_Er.py')


Instruction notebook has been successfully saved to 'Chuanyu_data_files/Guide_06_26_2024_5.ipynb'



In [7]:
find_Er_ED_transition = find_ED_transition(Er_energy)
find_Er_ED_transition 


For green (500.0 nm to 600.0 nm):
Number of selected transitions: 47

For red (620.0 nm to 750.0 nm):
Number of selected transitions: 67


{'green': {'E30(2H1_9_2)E14(2G1_7_2)': 502,
  'E26(4D_3_2)E8(4F_5_2)': 503,
  'E23(4D_7_2)E6(2H2_11_2)': 505,
  'E11(4G_11_2)E1(4I_13_2)': 506,
  'E29(4D_1_2)E12(4G_9_2)': 509,
  'E35(2F2_5_2)E28(2I_13_2)': 512,
  'E32(2H1_11_2)E15(2P_3_2)': 513,
  'E26(4D_3_2)E9(4F_3_2)': 514,
  'E20(2D1_5_2)E4(4F_9_2)': 516,
  'E29(4D_1_2)E13(2K_15_2)': 516,
  'E25(2L_17_2)E8(4F_5_2)': 518,
  'E6(2H2_11_2)E0(4I_15_2)': 520,
  'E22(4D_5_2)E6(2H2_11_2)': 521,
  'E28(2I_13_2)E10(2G1_9_2)': 523,
  'E29(4D_1_2)E14(2G1_7_2)': 523,
  'E25(2L_17_2)E9(4F_3_2)': 528,
  'E15(2P_3_2)E3(4I_9_2)': 530,
  'E34(2D2_3_2)E21(2H2_9_2)': 536,
  'E24(2I_11_2)E8(4F_5_2)': 537,
  'E19(4G_7_2)E4(4F_9_2)': 538,
  'E33(2F2_7_2)E21(2H2_9_2)': 538,
  'E23(4D_7_2)E7(4F_7_2)': 539,
  'E37(2G2_9_2)E32(2H1_11_2)': 539,
  'E5(4S_3_2)E0(4I_15_2)': 542,
  'E36(2G2_7_2)E29(4D_1_2)': 543,
  'E27(2D1_3_2)E10(2G1_9_2)': 544,
  'E24(2I_11_2)E9(4F_3_2)': 549,
  'E18(2P_1_2)E4(4F_9_2)': 554,
  'E21(2H2_9_2)E5(4S_3_2)': 555,
  'E17(4G_5_2)E4(

# Execution

In [8]:
from utils import *
from Point_Er import Point
from Lattice_Er import Lattice
from EnergyTransfer_Er import *
from Simulator_Er import Simulator

# [0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.5]
# [1, 5, 10, 20, 40, 80, 100, 200, 400, 800, 1000, 2000, 5000, 10000, 20000, 50000, 100000, 200000, 500000, 1000000, 2000000, 5000000, 10000000]

# [0.02, 0.04, 0.08, 0.16], Yb = 0.2
# [1, 5, 10, 20, 40, 80, 100, 200, 400, 800, 1000, 2000, 5000, 10000, 20000, 50000, 100000, 200000, 500000, 1000000, 2000000, 5000000, 10000000]


# Er_conc=[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]
# Er_conc=[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8]
# Er_conc=[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
# Er_conc=[0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.5]
# Yb_conc=0.2
# power_density =  [10, 20, 40, 80, 1*10**2, 2*10**2, 4*10**2, 8*10**2, 1*10**3, 2*10**3, 4*10**3, 8*10**3, 1*10**4, 2*10**4, 4*10**4, 8*10**4, 1*10**5, 2*10**5, 4*10**5, 8*10**5, 1*10**6, 2*10**6, 4*10**6, 8*10**6, 1*10**7]


# Er_conc=[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]
# Er_conc=[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8]
# Er_conc=[0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
# Er_conc=[0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.5]
# Yb_conc=0.2


# Er_conc = [0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.5]
# d_range = [8.0, 8.3, 8.0, 8.5, 8.0, 7.9, 8.2, 8.7]
# power_density = [1232.21547757,  1869.76441851,  2826.08782992,  4419.96018226,
#  7607.70488696, 10795.44959165, 13983.19429634, 17170.93900104,
#  20358.68370573, 23546.42841042, 26734.17311511, 29921.91781981]


# emitter_conc = [0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.5]
# power_density = [1.2*10**3, 1.8*10**3, 2.8*10**3, 4.4*10**3, 5*10**3, 7.5*10**3, 1.1*10**4, 1.4*10**4, 1.7*10**4, 2*10**4, 2.3*10**4, 2.7*10**4, 3*10**4]



# nanoparticle diameter: 8 nm, critical distance: 1 nm
# d=8
# shell=0
# r0=1



Er_conc = [0.04, 0.06, 0.08, 0.1, 0.12, 0.15, 0.2, 0.5]
power_density = [10, 20, 50, 100, 200, 500, 1000, 2000, 4000, 7000, 10000, 20000, 40000, 70000, 100000, 200000, 400000, 700000, 1000000, 2000000, 5000000, 10000000]



t1 = 7000
t2 = 10000



# Simulate and track progress

for i, c in enumerate(Er_conc):

    # print(i)
    # d=d_range[i]
    # print(d)

    if c not in myC:
        myC[c] = {}

    for p in power_density:
        if p not in myC[c]:
            print(f"Running simulation for Er concentration {c}, power density {p}")

            # sigma from MC paper, SI 
            tag_default['laser'] = 0.058 * p
            tag_default['laser_er'] = 0.0084 * p       
            
            my_lattice = Lattice(1-c, c, d, r0)  # shell
            my_simulator = Simulator(my_lattice, find_Er_ED_transition, tag=tag_default)
            result = my_simulator.simulate(t1, t2)  # r is a dictionary from 'sim_stats'

            # Store result and save progress
            myC[c][p] = result
            # store the current progress to the filepath
            save_progress(progress_filepath, myC)

            # Simulate time delay to represent real simulation time
            sleep(1)

print('\nAll progress have been finished.')


Running simulation for Er concentration 0.04, power density 10


100%|██████████| 3000/3000 [00:09<00:00, 331.39it/s]
/Users/liuchuan/Desktop/My local GitHub/Simulator_Er.py:711: RuntimeWarning: invalid value encountered in scalar divide
  sim_stats[f'{r_key_list[0]}_{r_key_list[1]}_ratio'] = np.mean(r_emission[r_key_list[0]]) / np.mean(r_emission[r_key_list[1]])
/Users/liuchuan/Desktop/My local GitHub/Simulator_Er.py:769: RuntimeWarning: invalid value encountered in scalar divide
  sim_stats[f'{color_totals_key_list[0]}_{color_totals_key_list[1]}_ratio_pop'] = color_totals[color_totals_key_list[0]] / color_totals[color_totals_key_list[1]]


Running simulation for Er concentration 0.04, power density 20


100%|██████████| 3000/3000 [00:09<00:00, 304.40it/s]


Running simulation for Er concentration 0.04, power density 50


...

Running simulation for Er concentration 0.1, power density 200000


100%|██████████| 3000/3000 [09:31<00:00,  5.25it/s]  


Running simulation for Er concentration 0.1, power density 400000


  3%|▎         | 225/7000 [00:15<07:41, 14.68it/s]


KeyboardInterrupt: 

# Save all data from progress file

In [9]:
# after all the progress are finished
# The first time to run the code, the folder 'Chuanyu_data_files' will have a new file: 'myC_05_06_2024_1.pkl', which store all data
# The second time to run the code, the folder 'Chuanyu_data_files' will have a new file: 'myC_05_06_2024_2.pkl', which store all data

from Er_save_file import *

saver = PickleSaver(base_name='myC', folder='Chuanyu_data_files')
saver.save_data(myC)

# # then, call the method with myC, to see the data structure
# data_structure = saver.load_all_data()

Data has been successfully saved to 'Chuanyu_data_files/myC_06_27_2024_1.pkl'



---
### If you want to run the code again, firstly make sure you have run the code above to store the data:

'''
saver = PickleSaver(base_name='myC', folder='Chuanyu_data_files')
saver.save_data(myC)b
all_save_data = saver.load_all_data()
'''
#### In the same directory, there would be a new file: myC_05_08_2024_1, which save all data from this simulation
#### Then you can detele the progress file and to run the code
---